In [2]:
import os
import sys
PROJECT_PATH = "/home/albert/Baikal-ML/" #insert your project path
sys.path.append(f"{PROJECT_PATH}")
from time import time

import torch

In [3]:
from nnetworks.layers.norm import MaskedBatchNorm1D

In [4]:
data = torch.rand((2,5,6))
data[0,1,:] = torch.tensor([1,2,3,4,5,6])
data[1,1,:] = torch.tensor([-6,-5,-4,-3,-2,-1])

In [5]:
data.mean((0,2)), data.var((0,2))

(tensor([0.4902, 0.0000, 0.3557, 0.6115, 0.4902]),
 tensor([ 0.1075, 16.5455,  0.1167,  0.1084,  0.1294]))

In [6]:
#mean = (data.sum(2, keepdim=True)/data.shape[2]).squeeze(2).mean(dim=0).view(1,-1,1)
#std = torch.sqrt((data-mean)**2).view(1,-1,1)

res = (data-data.mean((0,2), keepdim=True))/torch.sqrt(data.var((0,2), keepdim=True))
res

tensor([[[ 1.2048, -1.4850, -1.1499, -0.8790,  1.1564, -0.4733],
         [ 0.2458,  0.4917,  0.7375,  0.9834,  1.2292,  1.4751],
         [ 0.3415,  0.4865, -0.9577,  0.6017, -1.0097,  1.6982],
         [ 0.5957,  0.0629, -1.4953,  0.5890,  1.1120, -1.6756],
         [-0.9886, -0.8768,  0.3325,  1.1078, -0.5223,  1.1263]],

        [[ 1.0887, -0.2627,  0.2371, -0.5661, -0.2776,  1.4068],
         [-1.4751, -1.2292, -0.9834, -0.7375, -0.4917, -0.2458],
         [-0.9678, -0.5836, -0.8187,  1.0584,  1.1676, -1.0164],
         [ 0.8387, -0.7256,  0.9311,  0.2559,  0.6862, -1.1751],
         [-1.1633, -0.8321,  1.0702,  0.4876,  1.3362, -1.0775]]])

In [7]:
mask=torch.ones(data.shape[0],1,data.shape[2], dtype=bool)
valid_count = mask.sum(dim=2, keepdim=True) + 1e-5

# Compute mean and variance across valid positions only
mean = data.sum((0,2)) / valid_count.sum(0) # mean over length
mean = mean.view(1, -1, 1)
variance = ((data - mean) ** 2 * mask).sum((0,2)) / (valid_count.sum(0) - 1) # mean over length
variance = variance.view(1, -1, 1)

(data-mean)/variance.sqrt()

tensor([[[ 1.2048, -1.4850, -1.1499, -0.8790,  1.1564, -0.4733],
         [ 0.2458,  0.4917,  0.7375,  0.9834,  1.2292,  1.4751],
         [ 0.3415,  0.4865, -0.9577,  0.6017, -1.0097,  1.6982],
         [ 0.5957,  0.0629, -1.4953,  0.5890,  1.1120, -1.6756],
         [-0.9886, -0.8768,  0.3325,  1.1078, -0.5223,  1.1263]],

        [[ 1.0887, -0.2627,  0.2371, -0.5661, -0.2776,  1.4068],
         [-1.4751, -1.2292, -0.9834, -0.7375, -0.4917, -0.2458],
         [-0.9678, -0.5836, -0.8187,  1.0584,  1.1676, -1.0164],
         [ 0.8387, -0.7256,  0.9311,  0.2559,  0.6863, -1.1751],
         [-1.1633, -0.8321,  1.0702,  0.4876,  1.3362, -1.0775]]])

In [8]:
data.var((0,2), keepdim=True)

tensor([[[ 0.1075],
         [16.5455],
         [ 0.1167],
         [ 0.1084],
         [ 0.1294]]])

In [9]:
bn = MaskedBatchNorm1D(5, eps=0., momentum=0., affine=True, track_running_stats=True)
bn.bias, bn(data, mask=torch.ones(data.shape[0],1,data.shape[2], dtype=bool))

(Parameter containing:
 tensor([0., 0., 0., 0., 0.], requires_grad=True),
 tensor([[[ 1.2048, -1.4850, -1.1499, -0.8790,  1.1564, -0.4733],
          [ 0.2458,  0.4917,  0.7375,  0.9834,  1.2292,  1.4751],
          [ 0.3415,  0.4865, -0.9577,  0.6017, -1.0097,  1.6982],
          [ 0.5957,  0.0629, -1.4953,  0.5890,  1.1120, -1.6756],
          [-0.9886, -0.8768,  0.3325,  1.1078, -0.5223,  1.1263]],
 
         [[ 1.0887, -0.2627,  0.2371, -0.5661, -0.2776,  1.4068],
          [-1.4751, -1.2292, -0.9834, -0.7375, -0.4917, -0.2458],
          [-0.9678, -0.5836, -0.8187,  1.0584,  1.1676, -1.0164],
          [ 0.8387, -0.7256,  0.9311,  0.2559,  0.6862, -1.1751],
          [-1.1633, -0.8321,  1.0702,  0.4876,  1.3362, -1.0775]]],
        grad_fn=<AddBackward0>))

In [15]:
res_bn = torch.nn.BatchNorm1d(5, momentum=0., affine=False, track_running_stats=False)(data)
res_bn

tensor([[[ 1.2583, -1.5510, -1.2010, -0.9181,  1.2077, -0.4943],
         [ 0.2568,  0.5136,  0.7703,  1.0271,  1.2839,  1.5407],
         [ 0.3566,  0.5081, -1.0003,  0.6284, -1.0546,  1.7737],
         [ 0.6222,  0.0657, -1.5617,  0.6151,  1.1614, -1.7501],
         [-1.0325, -0.9157,  0.3472,  1.1570, -0.5455,  1.1763]],

        [[ 1.1371, -0.2744,  0.2476, -0.5912, -0.2900,  1.4693],
         [-1.5407, -1.2839, -1.0271, -0.7703, -0.5136, -0.2568],
         [-1.0107, -0.6095, -0.8551,  1.1054,  1.2195, -1.0616],
         [ 0.8759, -0.7578,  0.9725,  0.2673,  0.7167, -1.2273],
         [-1.2150, -0.8691,  1.1177,  0.5093,  1.3955, -1.1253]]])

In [13]:
res_bn = torch.nn.InstanceNorm1d(5, eps=0., momentum=0., affine=False, track_running_stats=True)(data)
res_bn

tensor([[[ 1.3790, -1.1343, -0.8212, -0.5681,  1.3337, -0.1890],
         [-1.4639, -0.8783, -0.2928,  0.2928,  0.8783,  1.4639],
         [ 0.1574,  0.3115, -1.2235,  0.4340, -1.2787,  1.5993],
         [ 0.6828,  0.1851, -1.2704,  0.6765,  1.1650, -1.4389],
         [-1.1606, -1.0332,  0.3449,  1.2285, -0.6292,  1.2496]],

        [[ 1.1116, -0.7257, -0.0461, -1.1380, -0.7459,  1.5441],
         [-1.4639, -0.8783, -0.2928,  0.2928,  0.8783,  1.4639],
         [-0.8286, -0.4175, -0.6691,  1.3395,  1.4565, -0.8807],
         [ 0.8720, -1.0671,  0.9866,  0.1496,  0.6831, -1.6243],
         [-1.1001, -0.7787,  1.0676,  0.5022,  1.3258, -1.0168]]])

In [18]:
x = torch.zeros((2,5))
x1 = x.clone()
x1[:,:] = 1 
x, x1

(tensor([[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]),
 tensor([[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]))